In [35]:
import pandas as pd
import numpy as np
import catboost as cb
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score

X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')

train_data = X_train.merge(y_train, on='ROW_ID')
test_data = pd.read_csv('X_test.csv')

In [36]:
def mle_params_all_from_group(g, r_cols, delta_t = 1.0):
    arr = g[r_cols].to_numpy(dtype = float)
    mean_r = arr.mean()
    var_r = arr.var(ddof = 0)
    sigma_hat = np.sqrt(var_r/delta_t)
    mu_hat = (mean_r / delta_t) + 0.5 * (sigma_hat**2)
    N_effective = arr.size
    return pd.Series({"mu_alloc": mu_hat, "sigma_alloc": sigma_hat, "N_effective": N_effective})

def compute_allocation_param(df, r_cols, delta_t = 1.0):
    out = (df.groupby ("ALLOCATION", observed = True)
           .apply(lambda g:mle_params_all_from_group(g, r_cols, delta_t))
           .reset_index()
           )
    return out

return_cols = [f'RET_{i}' for i in range(1, 21)]



In [37]:
alloc_train = compute_allocation_param(X_train, return_cols)
X_train = X_train.merge(alloc_train[['ALLOCATION', "mu_alloc", "sigma_alloc"]], on = "ALLOCATION", how = 'left')
train_data = X_train.merge(y_train, on = "ROW_ID")
alloc_test = compute_allocation_param(test_data, return_cols)
test_data = test_data.merge(alloc_test[['ALLOCATION', "mu_alloc", "sigma_alloc"]], on = "ALLOCATION", how = 'left')


C:\Users\Dell\AppData\Local\Temp\ipykernel_12380\2171627025.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g:mle_params_all_from_group(g, r_cols, delta_t))
C:\Users\Dell\AppData\Local\Temp\ipykernel_12380\2171627025.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g:mle_params_all_from_group(g, r_cols, delta_t))


In [38]:
def create_all_features(df):
    ret_cols = [f'RET_{i}' for i in range(1, 21)]
    df['ret_mean_5d'] = df[ret_cols[:5]].mean(axis=1) 
    df['ret_std_5d'] = df[ret_cols[:5]].std(axis=1) 
    df['ret_mean_20d'] = df[ret_cols].mean(axis=1) 
    df['momentum_ratio_5_20'] = df['ret_mean_5d'] / (df['ret_mean_20d'] + 1e-6) 
    rolling_std = df[ret_cols].T.rolling(window=3).std().T # une espece de volatilité sur fenetre glissante de 3 jours
    df['meta_volatility'] = rolling_std.std(axis=1) # la vola de cette vola : indice de stabilité
    df['interaction_ret_turnover'] = df['ret_mean_5d'] * df['AVG_DAILY_TURNOVER']
    return df

train_data = create_all_features(train_data)
test_data = create_all_features(test_data)


train_data = create_all_features(train_data)
test_data = create_all_features(test_data)




In [31]:
train_data.columns

Index(['ROW_ID', 'TS', 'ALLOCATION', 'RET_20', 'RET_19', 'RET_18', 'RET_17',
       'RET_16', 'RET_15', 'RET_14', 'RET_13', 'RET_12', 'RET_11', 'RET_10',
       'RET_9', 'RET_8', 'RET_7', 'RET_6', 'RET_5', 'RET_4', 'RET_3', 'RET_2',
       'RET_1', 'SIGNED_VOLUME_20', 'SIGNED_VOLUME_19', 'SIGNED_VOLUME_18',
       'SIGNED_VOLUME_17', 'SIGNED_VOLUME_16', 'SIGNED_VOLUME_15',
       'SIGNED_VOLUME_14', 'SIGNED_VOLUME_13', 'SIGNED_VOLUME_12',
       'SIGNED_VOLUME_11', 'SIGNED_VOLUME_10', 'SIGNED_VOLUME_9',
       'SIGNED_VOLUME_8', 'SIGNED_VOLUME_7', 'SIGNED_VOLUME_6',
       'SIGNED_VOLUME_5', 'SIGNED_VOLUME_4', 'SIGNED_VOLUME_3',
       'SIGNED_VOLUME_2', 'SIGNED_VOLUME_1', 'AVG_DAILY_TURNOVER', 'mu_alloc',
       'sigma_alloc', 'target', 'ret_mean_5d', 'ret_std_5d', 'ret_mean_20d',
       'momentum_ratio_5_20', 'meta_volatility', 'interaction_ret_turnover'],
      dtype='object')

In [47]:
################ TRAINING ################

train_data['y_binary'] = (train_data['target'] > 0).astype(int)

# features = les RET_i + les SIGNED_VOLUME_i + ds la partie FEATURES
original_ts_cols = [f'RET_{i}' for i in range(1, 21)]
features_to_use = [col for col in train_data.columns if col not in (['ROW_ID', 'target', 'y_binary'] )]

X = train_data[features_to_use].copy()
y = train_data['y_binary']
groups = train_data['TS']
X_test = test_data[features_to_use].copy()

X.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

print(features_to_use)
N_SPLITS = 20 # TO CHECK : à augmenter? 
gkf = GroupKFold(n_splits=N_SPLITS)
test_preds = []
oof_preds = np.zeros(len(X))

for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups=groups)):
    X_train, y_train = X.iloc[train_idx].copy(), y.iloc[train_idx].copy()
    X_val, y_val = X.iloc[val_idx].copy(), y.iloc[val_idx].copy()
    
    features_to_process = ['ret_mean_5d', 'momentum_ratio_5_20', 'interaction_ret_turnover', 'mu_alloc', 'sigma_alloc']
    for feature in features_to_process:
        market_map = X_train.groupby('TS')[feature].mean()
        # ici on va comparer X_train à toutes les lignes d'une meme TS 
        X_train[f'diff_from_market_{feature}'] = X_train[feature] - X_train['TS'].map(market_map)
        X_val[f'diff_from_market_{feature}'] = X_val[feature] - X_val['TS'].map(market_map)
        X_train[f'{feature}_rank'] = X_train.groupby('TS')[feature].rank(pct=True)
        X_val[f'{feature}_rank'] = X_val.groupby('TS')[feature].rank(pct=True)

    X_train.fillna(0, inplace=True)
    X_val.fillna(0, inplace=True)
    
    categorical_features = ['ALLOCATION', 'TS']
    # TO DO : Optuna ?
    model = cb.CatBoostClassifier(
        iterations=2000, learning_rate=0.015, depth=6, l2_leaf_reg=4,
        loss_function='Logloss', eval_metric='Accuracy',
        random_seed=fold, verbose=0, early_stopping_rounds=150
    )
    model.fit(X_train, y_train, cat_features=categorical_features, eval_set=(X_val, y_val))
    
    val_preds = model.predict(X_val)
    oof_preds[val_idx] = val_preds.flatten()
    print(f"Score du Fold {fold+1}: {accuracy_score(y_val, val_preds)}")
    
    # pareil pour X_test
    X_test_fold = X_test.copy()
    for feature in features_to_process:
        market_map = X_train.groupby('TS')[feature].mean()
        X_test_fold[f'diff_from_market_{feature}'] = X_test_fold[feature] - X_test_fold['TS'].map(market_map)
        X_test_fold[f'{feature}_rank'] = X_test_fold.groupby('TS')[feature].rank(pct=True)
    X_test_fold.fillna(0, inplace=True)

    preds = model.predict_proba(X_test_fold)[:, 1]
    test_preds.append(preds)

final_score = accuracy_score(y, oof_preds)
print(f"Score CV global sur {N_SPLITS} Folds : {final_score}")

['TS', 'ALLOCATION', 'RET_20', 'RET_19', 'RET_18', 'RET_17', 'RET_16', 'RET_15', 'RET_14', 'RET_13', 'RET_12', 'RET_11', 'RET_10', 'RET_9', 'RET_8', 'RET_7', 'RET_6', 'RET_5', 'RET_4', 'RET_3', 'RET_2', 'RET_1', 'SIGNED_VOLUME_20', 'SIGNED_VOLUME_19', 'SIGNED_VOLUME_18', 'SIGNED_VOLUME_17', 'SIGNED_VOLUME_16', 'SIGNED_VOLUME_15', 'SIGNED_VOLUME_14', 'SIGNED_VOLUME_13', 'SIGNED_VOLUME_12', 'SIGNED_VOLUME_11', 'SIGNED_VOLUME_10', 'SIGNED_VOLUME_9', 'SIGNED_VOLUME_8', 'SIGNED_VOLUME_7', 'SIGNED_VOLUME_6', 'SIGNED_VOLUME_5', 'SIGNED_VOLUME_4', 'SIGNED_VOLUME_3', 'SIGNED_VOLUME_2', 'SIGNED_VOLUME_1', 'AVG_DAILY_TURNOVER', 'mu_alloc', 'sigma_alloc', 'ret_mean_5d', 'ret_std_5d', 'ret_mean_20d', 'momentum_ratio_5_20', 'meta_volatility', 'interaction_ret_turnover']
Score du Fold 1: 0.5368013281682347
Score du Fold 2: 0.5158826784726065


KeyboardInterrupt: 

In [33]:
feature_importances = model.get_feature_importance()
importance_df = pd.DataFrame({
    'feature': model.feature_names_,
    'importance': feature_importances
}).sort_values('importance', ascending=False)

print(importance_df)

                                      feature  importance
0                                          TS   53.041904
44                                sigma_alloc   21.500952
1                                  ALLOCATION   19.069427
42                         AVG_DAILY_TURNOVER    6.387717
4                                      RET_18    0.000000
5                                      RET_17    0.000000
6                                      RET_16    0.000000
7                                      RET_15    0.000000
8                                      RET_14    0.000000
9                                      RET_13    0.000000
2                                      RET_20    0.000000
3                                      RET_19    0.000000
12                                     RET_10    0.000000
13                                      RET_9    0.000000
14                                      RET_8    0.000000
15                                      RET_7    0.000000
16            

In [34]:
final_predictions_proba = np.mean(test_preds, axis=0)
final_predictions = (final_predictions_proba > 0.5).astype(int)

submission = pd.DataFrame({'ROW_ID': test_data['ROW_ID'], 'prediction': final_predictions})

submission.to_csv('C:/Users/Dell/Desktop/update_submission_gbm_catboost.csv', index=False)